In [90]:
def max_memory(row):
    if row["method"] in ["Greedy","ThreeSieves","Random"]:
        return row["K"]
    elif row["method"] in ["SieveStreaming", "Salsa"]:
        epsilon = row["epsilon"]
        K = row["K"]
        lower = 1 # in all experiments we used m = 1
        upper = lower * K

        ilower = np.ceil(np.log(lower) / np.log(1+epsilon))
        ts = []
        t = np.power(1.0 + epsilon, ilower)
        while (t < upper):
            ts.append(t)
            ilower += 1
            t = np.power(1.0 + epsilon, ilower)

        if row["method"] == "SieveStreaming":
            return K*len(ts)
        else:
            return 3*K*len(ts)
    else:
        # See https://arxiv.org/pdf/1905.00948.pdf page 6 for memory computation
        epsilon = row["epsilon"]
        K = row["K"]

        mem = np.ceil(K*np.log(2) / epsilon)
        upper = np.log(K) / np.log(1+epsilon)
        i = 0
        while(i < upper):
            mem += K/np.power(1+epsilon,i)
            i += 1
        return mem
        
def relative_performance(row):
    K = row["K"]
    sigma = row["sigma"]
    reference = df.loc[ (df["method"] == "Greedy") & (df["K"] == K) & (df["sigma"] == sigma) ]
    perf = row["fval"] / reference["fval"]

    return perf.values[0]
    
def nice_name(row):
    if row["method"] in ["Greedy", "Random"]:
        return row["method"]
    elif row["method"] in ["SieveStreaming", "SieveStreaming++", "Salsa"]:
        #return "{} ε = {}".format(row["method"], row["epsilon"])
        return row["method"]
    else:
        #return "{} ε = {}, T = {}".format(row["method"], row["epsilon"], row["T"])
        return "{} T = {}".format(row["method"], int(row["T"]))

In [93]:
import pandas as pd
import numpy as np
import os
#import plotly.express as px

datasets = ["forestcover", "kddcup99", "creditfraud"] 
dfs = []

for d in datasets:
    df = pd.read_csv(os.path.join("experiments",d,"results.csv"))

    df["nice_name"] = df.apply(nice_name,axis=1)
    df["relative_performance"] = df.apply(relative_performance,axis=1)
    df["max_memory"] = df.apply(max_memory,axis=1)
    df["dataset"] = d
    dfs.append(df)

#df = pd.concat(dfs)
#df
#print(df)



In [94]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

#if not os.path.exists("images"):
    #os.mkdir("images") 

pio.orca.config.use_xvfb = True

# TODO COLOR MAPPING WITH NAME
eps = 0.001
fig = make_subplots(rows=3, cols=len(dfs), subplot_titles=[str(df["dataset"].unique()[0]) for df in dfs], horizontal_spacing = 0.05, vertical_spacing = 0.05)

for i, df in enumerate(dfs):
    sigmas = df["sigma"].unique()
    dff = df.loc[ df["method"] != "Greedy"]

    if i == 0:
        showlegend = True
    else:
        showlegend = False

    dff = dff.loc[ (dff["sigma"] == sigmas[0]) & ((dff["epsilon"].isnull()) | (dff["epsilon"] == eps))]
    for name in dff["nice_name"].unique():
        df_method = dff.loc[ dff["nice_name"] == name]
        
        fig.add_trace(go.Scatter(x = df_method["K"], y = df_method["relative_performance"], name = name, showlegend = showlegend), row = 1, col = i+1)
        fig.add_trace(go.Scatter(x = df_method["K"], y = df_method["runtime"], name = name, showlegend = False), row = 2, col = i+1)
        fig.add_trace(go.Scatter(x = df_method["K"], y = df_method["max_memory"], name = name, showlegend = False), row = 3, col = i+1)
    

fig.update_xaxes(title_text="K", row=len(dfs)+1, col=1, title_font = {"size": 12})
fig.update_xaxes(title_text="K", row=len(dfs)+1, col=2, title_font = {"size": 12})

fig.update_yaxes(title_text="Performance", row=1, col=1, title_standoff = 25, title_font = {"size": 12})
fig.update_yaxes(title_text="Runtime [s]", row=2, col=1, title_font = {"size": 12})
fig.update_yaxes(title_text="Max. Memory", row=3, col=1, title_font = {"size": 12})

for i, df in enumerate(dfs):
    fig.update_yaxes(row=2, col=i+1, type="log")
    fig.update_yaxes(row=3, col=i+1, type="log")


fig.update_layout(legend=dict(
    orientation="h",yanchor="bottom",y=-0.1,
    xanchor="right",x=1
),height=1200, width=1200)
#fig['layout'].update(height=1000, width=1200)
fig.show()

# fig.write_image("images/{}_{}_{}.pdf".format("Accuracy",mtype.replace(" ", "-"), ne))